## Book Recommender System using Clustering (Collaborative based)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Data Collection

In [2]:
books = pd.read_csv('dataset/BX-Books.csv', sep = ';',error_bad_lines = False, encoding = 'latin-1')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.p

In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
books.shape

(271360, 8)

In [5]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [6]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-M']]
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-M
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [7]:
books.rename(columns = {
    "ISBN":"isbn",
    "Book-Title": "title",
    "Book-Author":"author",
    "Year-Of-Publication": "year",
    "Publisher": "publisher",
    "Image-URL-M": "image_url",}, inplace = True)

In [8]:
books.head()

,isbn,title,author,year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [9]:
users = pd.read_csv('dataset/BX-Users.csv', sep = ';',error_bad_lines = False, encoding = 'latin-1')
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [10]:
users.shape

(278858, 3)

In [11]:
ratings =  pd.read_csv('dataset/BX-Book-Ratings.csv', sep = ';',error_bad_lines = False, encoding = 'latin-1')
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [12]:
ratings['Book-Rating'].max()

10

In [13]:
ratings.shape

(1149780, 3)

In [14]:
ratings.rename(columns = {
    "User-ID":"user_id",
    "ISBN": "isbn",
    "Book-Rating": "rating"}, inplace = True)

ratings.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [15]:
print('Book data: ', books.shape)
print('User data: ', users.shape)
print('Rating data: ' ,ratings.shape)

Book data:  (271360, 6)
User data:  (278858, 3)
Rating data:  (1149780, 3)


### Data Analysis

In [16]:
## the total number of users who engaged in the book rating exercise
print(" Total number of users who rated one or more books:" , ratings['user_id'].nunique())

 Total number of users who rated one or more books: 105283


In [17]:
## Check number of books each user has rated
ratings['user_id'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
271728        1
245123        1
234886        1
259466        1
187812        1
Name: user_id, Length: 105283, dtype: int64

In [18]:
## obtain users who have rated 200 books or more
target_users = ratings['user_id'].value_counts()>=200
print('target users with >= 200 ratings: ', target_users.sum())

target users with >= 200 ratings:  905


In [19]:
## get the target user_id
target_users_id = target_users[target_users].index

In [20]:
## Ratings data with target users: Extract target users
ratings = ratings[ratings['user_id'].isin(target_users_id)]

In [21]:
ratings.head()

,user_id,isbn,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [22]:
ratings.reset_index(inplace = True)
ratings.head()

,index,user_id,isbn,rating
0,1456,277427,002542730X,10
1,1457,277427,0026217457,0
2,1458,277427,003008685X,8
3,1459,277427,0030615321,0
4,1460,277427,0060002050,0


In [23]:
ratings.drop(columns = 'index',axis = 1,inplace = True)
ratings.head()

,user_id,isbn,rating
0,277427,002542730X,10
1,277427,0026217457,0
2,277427,003008685X,8
3,277427,0030615321,0
4,277427,0060002050,0


In [24]:
## merge the book data and rating data on isbn
ratings_with_books = ratings.merge(books, on = 'isbn' )

In [25]:
ratings_with_books.head()

,user_id,isbn,rating,title,author,year,publisher,image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [26]:
ratings_with_books.shape

(488742, 8)

In [27]:
## number of ratings per book
num_ratings = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [28]:
num_ratings.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [29]:
num_ratings.rename(columns = {"rating":"num_of_ratings"},inplace=True)
num_ratings.head()

,title,num_of_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [30]:
## merge num_rating with ratings_with_books
final_rating = ratings_with_books.merge(num_ratings, on = 'title')

In [31]:
final_rating.head()

,user_id,isbn,rating,title,author,year,publisher,image_url,num_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [32]:
final_rating.shape

(488742, 9)

In [33]:
## get books with number of rating = 50 or more
final_rating = final_rating[final_rating['num_of_ratings']>=50]

In [34]:
final_rating.sample(10)

,user_id,isbn,rating,title,author,year,publisher,image_url,num_of_ratings
32831,172030,0425145638,0,Fatal Cure,Robin Cook,1997,Berkley Publishing Group,http://images.amazon.com/images/P/0425145638.0...,60
32850,265313,0425145638,0,Fatal Cure,Robin Cook,1997,Berkley Publishing Group,http://images.amazon.com/images/P/0425145638.0...,60
78683,259260,1558744150,9,Chicken Soup for the Woman's Soul (Chicken Sou...,Jack Canfield,1996,Health Communications,http://images.amazon.com/images/P/1558744150.0...,68
94616,129008,0786890436,0,The Diary of Ellen Rimbauer: My Life at Rose Red,Ellen Rimbauer,2002,Hyperion Press,http://images.amazon.com/images/P/0786890436.0...,65
31165,60244,0385492081,0,Into Thin Air : A Personal Account of the Mt. ...,JON KRAKAUER,1998,Anchor,http://images.amazon.com/images/P/0385492081.0...,127
41275,200226,051512608X,0,The Reef,Nora Roberts,1999,Jove Books,http://images.amazon.com/images/P/051512608X.0...,70
186970,174304,0671042858,0,The Girl Who Loved Tom Gordon,Stephen King,2000,Pocket,http://images.amazon.com/images/P/0671042858.0...,70
8078,230522,0446350982,0,Presumed Innocent,Scott Turow,1988,Warner Books,http://images.amazon.com/images/P/0446350982.0...,149
26343,66680,0345422317,0,The Genesis Code,John Case,1998,Ballantine Books,http://images.amazon.com/images/P/0345422317.0...,68
19753,82831,0440222656,0,The Horse Whisperer,Nicholas Evans,1996,Dell,http://images.amazon.com/images/P/0440222656.0...,213


In [35]:
final_rating.shape

(62149, 9)

In [36]:
## drop duplicates
final_rating.drop_duplicates(['user_id','title'],inplace =True)

In [37]:
final_rating.shape

(60141, 9)

In [38]:
##create pivot table
book_pivot = final_rating.pivot_table(columns = 'user_id', index = 'title', values = 'rating')

In [39]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [40]:
book_pivot.shape

(746, 894)

In [41]:
book_pivot.fillna(0,inplace = True)

In [42]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
from scipy.sparse import csr_matrix

In [44]:
book_sparse = csr_matrix(book_pivot)

In [45]:
book_sparse

<746x894 sparse matrix of type '<class 'numpy.float64'>'
	with 15040 stored elements in Compressed Sparse Row format>

### Training 

In [46]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm = 'brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [47]:
distance , suggestion = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1,-1), n_neighbors=6)

In [48]:
distance

array([[ 0.        , 68.78953409, 70.        , 72.64296249, 76.83098333,
        77.28518616]])

In [49]:
suggestion

array([[237, 240, 238, 241, 184, 539]], dtype=int64)

In [50]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [51]:
## get books name
books_name = book_pivot.index

### Save helper files 

In [52]:
import pickle
pickle.dump(model, open('helperFiles/model.pkl', 'wb'))
pickle.dump(books_name, open('helperFiles/books_name.pkl', 'wb'))
pickle.dump(final_rating, open('helperFiles/final_rating.pkl', 'wb'))
pickle.dump(book_pivot, open('helperFiles/book_pivot.pkl', 'wb'))

### Helper Function to recommend book

In [53]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance , suggestion = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1,-1), n_neighbors=6)
    
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [54]:
book_name = 'No Safe Place'
recommend_book(book_name)

No Safe Place
Long After Midnight
A Civil Action
Exclusive
Table For Two
Lake Wobegon days


In [55]:
book_pivot.index[0]

'1984'